In [2]:
import pickle
import pandas as pd
import numpy as np
import re

In [3]:
train = pd.read_pickle('train_clean.pkl')
test = pd.read_pickle('test_clean.pkl')

In [4]:
train.head()

,id,title,text_clean
0,619941,Loch Katrine,\n\n\n\n\nInfobox lake\n name Loch Katrine\n ...
1,3884222,Bhadayasa,\n\nInfobox royalty\n image FileBhadrayasha c...
2,4229879,Lee Jones (author),\n\nLee Jones is an online poker executive and...
3,5320685,School District 54 Bulkley Valley,Infobox school district\n name Scho...
4,9146365,Combing,Interlace disambiguationInterlacethe combing o...


In [5]:
test.head()

,id,title,text_clean
0,2936718,Meitei Christians,\n\n\nThe Meitei Christians are a Christianity...
1,17023672,Musical expression,\n\n\n\n\nMusical expression is the art of pla...
2,37590035,READ 180,\nREAD is a reading intervention program in w...
3,3191002,The Ambulance,\nInfobox film\n name The Ambulanc...
4,24154353,Sifo Company,\n\nThe Sifo Company which did business as Sif...


In [6]:
print test.shape, train.shape

(1578, 3) (4379, 3)


In [7]:
#Remove non-letters & split into words       
train['text_clean'] = train['text_clean'].map(lambda x: re.sub("[^a-zA-Z]", " ", x).lower().split())
test['text_clean'] = test['text_clean'].map(lambda x: re.sub("[^a-zA-Z]", " ", x).lower().split())
test.head()

,id,title,text_clean
0,2936718,Meitei Christians,"[the, meitei, christians, are, a, christianity..."
1,17023672,Musical expression,"[musical, expression, is, the, art, of, playin..."
2,37590035,READ 180,"[read, is, a, reading, intervention, program, ..."
3,3191002,The Ambulance,"[infobox, film, name, the, ambulance, image, t..."
4,24154353,Sifo Company,"[the, sifo, company, which, did, business, as,..."


In [8]:
from nltk.corpus import stopwords # Import the stop word list
print stopwords.words("english") 

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [9]:
stops = set(stopwords.words("english")) 

In [11]:
#remove stop words and words with length <= 2
train['text_clean'] = train['text_clean'].map(lambda x: " ".join([w for w in x if not w in stops and len(w)>2]))
test['text_clean'] = test['text_clean'].map(lambda x: " ".join([w for w in x if not w in stops and len(w)>2]))


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [91]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 9000) 

In [92]:
train_data_features = vectorizer.fit_transform(train['text_clean'])

In [93]:
train_data_features = train_data_features.toarray()

In [94]:
vocab = vectorizer.get_feature_names()
print vocab

[u'aaa', u'aaron', u'aba', u'abandon', u'abandoned', u'abbas', u'abbey', u'abbreviation', u'abc', u'abd', u'abdul', u'abdullah', u'abilities', u'ability', u'abkhaz', u'abkhazia', u'able', u'aboard', u'abolished', u'aboriginal', u'abortion', u'abraham', u'abroad', u'absence', u'absent', u'absolute', u'absolutely', u'absorbed', u'abstract', u'abu', u'abuse', u'abused', u'abuses', u'academia', u'academic', u'academics', u'academy', u'accelerator', u'accept', u'acceptable', u'acceptance', u'accepted', u'accepting', u'access', u'accessdate', u'accessdatejanuary', u'accessdatemarch', u'accessed', u'accessible', u'accident', u'accidents', u'acclaimed', u'accommodate', u'accommodation', u'accompanied', u'accompanying', u'accomplished', u'accord', u'accordance', u'according', u'accordingly', u'account', u'accountability', u'accounting', u'accounts', u'accreditation', u'accredited', u'accuracy', u'accurate', u'accusations', u'accused', u'accusing', u'ace', u'achieve', u'achieved', u'achievement'

In [95]:
feature_matrix = pd.DataFrame(train_data_features, index = train['title'], columns=vectorizer.get_feature_names())
print feature_matrix.shape

(4379, 9000)


In [19]:
pov = pd.read_csv('POV.csv')
pov.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
pov.head()

,id,title
0,5597,'Politics of Cyprus'
1,14668,'Economy of Iraq'
2,24400,'Pair programming'
3,32783,'Antisemitism and the New Testament'
4,38424,'Bikram Yoga'


In [20]:
pov['title_strip'] = pov['title'].map(lambda x: re.sub("'", '', x))
pov.head()

,id,title,title_strip
0,5597,'Politics of Cyprus',Politics of Cyprus
1,14668,'Economy of Iraq',Economy of Iraq
2,24400,'Pair programming',Pair programming
3,32783,'Antisemitism and the New Testament',Antisemitism and the New Testament
4,38424,'Bikram Yoga',Bikram Yoga


In [21]:
#add labels to train set
train['pov'] = train['title'].isin(pov['title_strip'])
test['pov'] = test['title'].isin(pov['title_strip'])

test.head()

,id,title,text_clean,pov
0,2936718,Meitei Christians,meitei christians christianity movement based ...,True
1,17023672,Musical expression,musical expression art playing singing music e...,True
2,37590035,READ 180,read reading intervention program wide use stu...,True
3,3191002,The Ambulance,infobox film name ambulance image ambulancejpg...,False
4,24154353,Sifo Company,sifo company business sifos toys sifo novelty ...,False


In [ ]:
#join features with labels
#feature_matrix = pd.concat([feature_matrix, pd.DataFrame({'pov': list(df['pov'])}, index=df['title'])], axis=1)

In [101]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C=.5) 

logit = logit.fit(feature_matrix, train['pov'])

In [97]:
# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(test['text_clean'])
test_data_features = test_data_features.toarray()


In [102]:
print "test score:", logit.score(test_data_features, test['pov'])
print "train score:", logit.score(train_data_features, train['pov'])

test score: 0.880861850444
train score: 0.99954327472


In [65]:
logit.score(test_data_features[test['pov']],[True]*727)

0.90646492434662995

In [66]:
logit.score(test_data_features[test['pov']==False],[False]*851)

0.8683901292596945